In [1]:
import pandas as pd
import numpy as np
#branch loda
dataset_path = "../dataset/mlcourse-dota2-win-prediction/train_features.csv"

In [9]:
df = pd.read_csv(dataset_path)

print(df.columns)

print(np.unique(df["game_mode"]))
print(len(df["game_mode"]))
print(len(df[df["game_mode"]==4]))
df_22 = df[df["game_mode"]==22]
print(len(df[df["game_mode"]==2]))
df_2 = df[df["game_mode"]==2]

print(len(df_22[df_22["lobby_type"]==7]))

print(len(df_22[df_22["lobby_type"]==0]))


print(len(df_2[df_2["lobby_type"]==7]))

print(len(df_2[df_2["lobby_type"]==0]))

Index(['match_id_hash', 'game_time', 'game_mode', 'lobby_type',
       'objectives_len', 'chat_len', 'r1_hero_id', 'r1_kills', 'r1_deaths',
       'r1_assists',
       ...
       'd5_stuns', 'd5_creeps_stacked', 'd5_camps_stacked', 'd5_rune_pickups',
       'd5_firstblood_claimed', 'd5_teamfight_participation',
       'd5_towers_killed', 'd5_roshans_killed', 'd5_obs_placed',
       'd5_sen_placed'],
      dtype='object', length=246)
[ 2  3  4  5 12 16 22 23]
39675
3564
408
25795
5967
379
29
29
29
['d1_hero_id', 'd1_kills', 'd1_deaths', 'd1_assists', 'd1_denies', 'd1_gold', 'd1_lh', 'd1_xp', 'd1_health', 'd1_max_health', 'd1_max_mana', 'd1_level', 'd1_x', 'd1_y', 'd1_stuns', 'd1_creeps_stacked', 'd1_camps_stacked', 'd1_rune_pickups', 'd1_firstblood_claimed', 'd1_teamfight_participation', 'd1_towers_killed', 'd1_roshans_killed', 'd1_obs_placed', 'd1_sen_placed']
['r1_hero_id', 'r2_hero_id', 'r3_hero_id', 'r4_hero_id', 'r5_hero_id', 'd1_hero_id', 'd2_hero_id', 'd3_hero_id', 'd4_hero_id', 

In [5]:
print(df)

                          match_id_hash  game_time  game_mode  lobby_type  \
0      a400b8f29dece5f4d266f49f1ae2e98a        155         22           7   
1      b9c57c450ce74a2af79c9ce96fac144d        658          4           0   
2      6db558535151ea18ca70a6892197db41         21         23           0   
3      46a0ddce8f7ed2a8d9bd5edcbb925682        576         22           7   
4      b1b35ff97723d9b7ade1c9c3cf48f770        453         22           7   
...                                 ...        ...        ...         ...   
39670  defd0caeed6ea83d7d5fbdec013fe7d1       1783         22           0   
39671  bc7a87ed5f9c2bca55f9f7a93da0b0c5        377         22           7   
39672  e2ca68ac1a6847f4a37f6c9c8ee8695b        643         22           7   
39673  47ad6454ede66c1c78fdaa9391dfc556       2405         22           7   
39674  9928dfde50efcbdb2055da23dcdbc101       1775         22           0   

       objectives_len  chat_len  r1_hero_id  r1_kills  r1_deaths  r1_assist